In [1]:
import os
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-5-mini",
    messages=[
        {"role": "user", "content": "Hello!"}
    ]
)

print(response.choices[0].message.content)

Hi there! How can I help you today — a question, writing or coding help, planning something, or something else?


## LLM CALL 


In [4]:
from langchain_openai import ChatOpenAI
llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

llm_openai.invoke("Bro, tell me a fun fact").content

"Bro — fun fact: wombats poop in perfect little cubes.  \n\nTheir intestines shape the poop into cubes so it doesn't roll away, which helps them mark territory and communicate. Wildly practical and oddly adorable. Want another one?"

### USING init_chat_model  a factory function that automatically initializes the correct chat model class based on the provider you specify, without needing to import provider-specific classes

In [7]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5-mini", model_provider="openai")

llm.invoke("Bro, tell me a fun fact").content


'Bro — fun fact: wombats poop in perfect little cubes. They do it to stack the piles as territorial/attracting markers, and their uniquely-shaped, very slow-moving intestines are what mold the poop into cubes. Want another weird one?'

# MESSAGES

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(model="gpt-5-mini", temperature=0)

my_message = [
    HumanMessage(content="Bro, tell me a fun fact?"),
    SystemMessage(content="You are a gen-z assistance. who always answer in afun way")
]

llm_openai.invoke(my_message).content

'Bro, wild fact: bananas are technically berries, but strawberries aren’t. 🍌🤯\n\nBotanical tea: a “berry” comes from one ovary and has seeds inside — bananas check that box. Strawberries are actually aggregate fruits with the seeds on the outside. Mind = blown. Wanna another weird one?'

In [ ]:
from langchain_openai import ChatOpenAI
import os
from load_dotenv import load_dotenv
load_dotenv()

if os.environ.get("OPENAI_API_KEY"):
    print("API KEY VARAIBLE EXIST")

SyntaxError: invalid syntax (1893176779.py, line 3)